# Multi-hospital Organ Transplants

This notebook explores expectations related to our organ transplant scenario. If you have not yet done so, please refer to **AgentsAndEvents.ipynb** for important events and fluents used in this scenario, and **Expectations.ipynb** for a discussion of expectations in this context. Both of these notebooks are located in the parent directory, **MultipleAgents**.

### Set up

We need to set up our environment by loading in our event and fluent declarations from **dec:notation.pl** and **AE.pl**. The latter contains the code from **AgentsAndEvents.ipynb** the last time that notebook was run.

<font color='red'>IMPORTANT NOTE: you should run **AgentsAndEvents.ipynb** prior to running this notebook to ensure that **AE.pl** contains up to date predicates! You must run the notebook, not just update and save it.</font>

In [1]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).
?- ['AE'].

true.
false.
true.
true.

## Compatibility / Suitability

<font color='red'> NB: This notebook is currently incomplete. Work needs to be done to allow for general incompatibility or compatibility fluent queries.</font>

An agent may expect that when it requests a resource, and its request is matched to a resource offer from another agent, that the match is "compatible". In the organ transplant scenario, the recipient's hospital might expect that any organ offered to the recipient by the monitoring system is compatible with the recipient in terms of blood type, medical conditions and any other important factors. Similarly, the donor's hospital might expect that the recipient of their offered organ is a suitable recipient, such that the organ is not wasted. 

There are two types of compatibility to consider here - the compatibility between the donor and the recipient, and compatibility between the type of organ and properties of either the recipient or the donor. An example of the former is whether the donor has a blood type suitable for the recipient. An example of the latter is whether the donor is too old to donate their heart. Note that this case does not consider any property of the recipient.

### Compatibility fluents

We can declare compatibility (and incompatibility) rules for determining whether patients are suitable to be potential donors or recipients and for matching an organ donor to a recipient. The `newCompatRule` is used to initiate new compatibility rules, while `newIncompatRule` initiates new incompatibility rules. Each terminates the opposing compatibility rule. This may require some tweaking if we wish opposing rules to cause a conflict violation rather than overwriting an earlier rule.

For donor-patient matching, we wish to differentiate between known compatibility or incompatibility and unknown compatibility. For instance, if we know that severe asthma makes lungs unsuitable for organ donation, but does not affect other organs, the compatibility of organs from a patient with severe asthma to a potential recipient should be treated differently to the compatibility of organs from a patient with Parkinson's disease, if the effect of this condition is not defined by any existing compatibility rules.

Compatibility rules have the arguments `Situation`, `Property1` and `Property2`. 
- `Situation` refers to the type of compatibility rule - `recipient`, `donor` or `transplant`. `recipient` and `donor` rules can be viewed as **suitability** rules. A `donor` rule states whether various properties of a potential donor make them suitable to donate. `donor` and `recipient` rules will only take into account the organ in question and details about the patient of interest, and will not rely on information about the other party. For example, we could have a `donor` rule stating that heart donors must not be aged over 65. This rule does not involve any details about a potential recipient. Likewise, a `recipient` rule states whether a recipient is eligible to receive a donation, without involving any details about the potential donor. A `transplant` rule combines properties of the donor and the recipient to determine whether the two parties are compatible for a donation. 
- *`Property1` and `Property2` each have the format `Type:Value` and detail two properties which we wish to check for compatibility. Examples of properties are blood type, age, medical condition and organ. In the `transplant` situation, `Property1` is a donor property and `Property2` is a recipient property.

As our compatibility rules may change over time, they are encoded as fluents. An example in which compatibility rules change over time, is the suitability of organs from donors with Hepatitis B and C or HIV. In the past, these organs were not suitable for donation (except potentially to recipients already diagnosed with the disease). However, a [recent study has shown](https://www.npr.org/sections/health-shots/2019/04/03/709533047/hepatitis-c-not-a-barrier-for-organ-transplantation-study-finds) that if recipients are given antiviral drugs, this can prevent them from getting the disease from an infected organ. 

Before such a study is recognised by a medical institution, the hospital may have a compatibility rule stating that Hepatitis B, Hepatitis C and HIV patients cannot donate to recipients without the disease(s). Once such a study is approved, the hospital may wish to change the compatibility rule to state that such donations are acceptable provided that an appropriate antiviral drug treatment is undertaken by the recipient.

All of our agents may make use of compatibility rules (although not necessarily the same ones), so we declare our *initiates* and *terminates* clauses irrespective of the agent.

In [2]:
% File: compatibilityRules.pl
initiates(_:newCompatRule(Situation, Property1, Property2), compatRule(Situation, Property1, Property2),_).
terminates(_:newIncompatRule(Situation, Property1, Property2), compatRule(Situation, Property1, Property2),_).
initiates(_:newIncompatRule(Situation, Property1, Property2), incompatRule(Situation, Property1, Property2),_).
terminates(_:newCompatRule(Situation, Property1, Property2), incompatRule(Situation, Property1, Property2),_).

### Example Rules

We set our initial compatibility rules to apply for *monSys*, as it can check whether a donor and a recipient are compatible when a match occurs. Currently, compatibility rules have been declared for organs, medical conditions, blood types, ages, and combinations of these categories. The organs used and age compatibility rules are based on information from [Organ Donation New Zealand](https://www.donor.co.nz/facts-and-myths/). Blood type compatibility is based on information from [UC Davis Health](https://health.ucdavis.edu/transplant/livingkidneydonation/matching-and-compatibility.html.). We also declare a medical conditon compatibility rule stating that a donor and recipient are compatible if the donor and the recipient have the same condition (and the donor has no other conditions not present in the recipient).

For categorical properties, we want to know which values belong in our category. For example, when checking blood types, we want to know that we recognise types A, B, AB and O. The `propertyList/2` fluents have the arguments `Type` and `List`, where `Type` refers to the property category, such as `bloodType`, and `List` is a list of the recognised values of that type. For example, the recognised blood types are defined below:
- `initially(monSys:propertyList(bloodType, ["A", "AB", "B", "O"])).`

While it may be unlikely that any new blood types are about to be discovered, given that the existing categories were determined over a century ago, it is best practice to design a system that can cope with changes if necessary. A possible way to achieve a new blood type may be a mutation, or widening our organ donation system to animal organ transplants. This is why `propertyList/2` is a fluent and not a standard predicate.

When we have defined the acceptable values of a property type, we are able to infer from compatibility rules when a property combination is incompatible, or likewise from an incompatibility rule we can infer compatible combinations. For example, `compatRule(transplant, bloodType:"A", bloodType:["A", "AB"])` states that if the donor has blood type A, their organs can be donated to patients who have blood type A or AB. As the four types of blood are initially declared to be A, B, AB and O, we can then infer that such a donor cannot donate to patients with blood type B or O.

Our compatibility rule properties have the format `Property:Value`. `Value` can be a single value or a list of values. Also supported is values with the form `(min:MinimumValue, max:MaximumValue)` for checking if a value is within a given range.

In [3]:
% File: initialRules.pl
% Recognised values in property categories
initially(monSys:propertyList(organ, ["Heart","Kidney", "Liver","Lungs", "Pancreas"])).
initially(monSys:propertyList(condition, ["HIV", "Hep-B", "Hep-C", "Heart Disease", "Severe Asthma", "Chronic Pulmonary Disease"])).
initially(monSys:propertyList(bloodType, ["A", "AB", "B", "O"])).
% Compatibility rules for transplants
initially(monSys:compatRule(transplant, bloodType:"A", bloodType:["A", "AB"])).
initially(monSys:compatRule(transplant, bloodType:"B", bloodType:["B", "AB"])).
initially(monSys:compatRule(transplant, bloodType:"AB", bloodType:"AB")).
initially(monSys:compatRule(transplant, bloodType:"O", bloodType:["A","AB","B","O"])).
initially(monSys:compatRule(transplant, condition:X, condition:X)).
% Suitability rules
initially(monSys:compatRule(donor, organ:"Heart", age:(min:0, max:65))).
initially(monSys:compatRule(donor, organ:"Lungs", age:(min:0, max:70))).
initially(monSys:compatRule(donor, organ:"Liver", age:_)).
initially(monSys:compatRule(donor, organ:"Pancreas", age:(min:0, max:45))).
initially(monSys:compatRule(donor, organ:"Kidney", age:_)).

initially(monSys:incompatRule(donor, organ:"Liver", condition:"Heart Disease")).

### Helper predicates

In order to simplify predicate rules, we declare a helper function which allows us to either select a singular argument or get arguments from a list

In [4]:
% File: extract.pl
extract(Input, Result):- is_list(Input), member(Result, Input).
extract(Input, Input):- \+ is_list(Input).

Some of our example compatibility rules use age ranges. In order to calculate the current age of a patient from their date of birth, we consult **date_time.pl**, which contains code from the **Date Time** package by Falco Nogatz (see https://github.com/fnogatz/date_time). The Deepnote kernel does not allow us to install this package using **pack_install/1**. Note that Deepnote uses UTC time, which is 13 hours behind New Zealand Daylight Time (September - April) and 12 hours behind New Zealand Standard Time (April - September). Hence, the current date used may be one day earlier than expected if the notebook is run before 1pm.

In [5]:
?- ['date_time'].

true.

The **transform/2** predicate allows us to declare the linkage between properties without having to rewrite our compatibility rules when the format of patient information changes. The **transform** predicate uses the **convert/2** predicate to check if our property can be written in a different form. If a convert rule has been established for the input, the converted output is used by the transform predicate. Otherwise, the transform predicate produces the input without any changes.

As an example, we declare that dates of birth can be converted to ages using the **date_age/2** predicate from the Date Time pack:
- `convert(dob:(Day, Month, Year), age:Age):- date_age(date(Year, Month, Day), Age).`
If hospitals changed to entering birth dates using American format, with Months first, then Days, then Years, we could make a small change to our convert rule to support this, rather than having to change multiple parts of our compatibility logic.

In [6]:
% File: transformations.pl
% Dates of birth should be converted into ages based on the current date
convert(dob:(Day, Month, Year), age:Age):-  \+ var(Day), \+ var(Month), \+ var(Year), date_age(date(Year, Month, Day), Age).
transform(X,Y):- convert(X,Y), !.
transform(X,X):- \+ convert(X,_).

### Obtaining compatibility predicates from rules

From our compatibility rules, we wish to establish resulting compatibility fluents. These fluents do not have `initiate` or `terminate` clauses associated with them, and are based on the current compatibility rules.

In order to find out if two properties are compatible in a given situation, we check to see whether the properties can be transformed into another form. We then check if there is a compatibility rule which involves the given property types. If one of our properties has a numerical value, we check that it is within the range specified in our compatibility rule. If our compatibility rule property is provided as a list, we check if the property that we are interested in is a member of that list.

Once we have established if `Property1` and `Property2` are compatible, we can determine that `Property1` is not compatible with any other properties of the same type as `Property2` which are not included in an appropriate compatibility rule. The same logic applies to using incompatibility rules to determine incompatible matches, and any remaining compatible matches.

In [7]:
% File: resultingCompatibilities.pl

% Compatibility from compatibility rules
holdsAt(Agent, compat(Situation, Type1:Value1, Type2:Value2), T):- transform(Type1:Value1, Type1Convert:Value1Convert),  % transform properties if applicable
transform(Type2:Value2, Type2Convert:Value2Convert), 
holdsAt(Agent, compatRule(Situation, Type1Convert:RuleValue1, Type2Convert:RuleValue2),T),                   % applicable compatibility rule holds
(\+var(RuleValue1), (RuleValue1 = (min:Min, max:Max)) ->  
(Value1Convert >= Min, Value1Convert =< Max); extract(RuleValue1, Value1Convert)),                           % Apply numerical range or extract from list
(\+var(RuleValue2), (RuleValue2 = (min:Min, max:Max)) ->  
(Value2Convert >= Min, Value2Convert =< Max); extract(RuleValue2, Value2Convert)), !.                        % Repeat for property 2

% Incompatibility from compatibility rules
holdsAt(Agent, incompat(Situation, Type1:Value1, Type2:Value2), T):- 
transform(Type1:Value1, Type1Convert:Value1Convert), transform(Type2:Value2, Type2Convert:Value2Convert),    % transform properties if applicable
holdsAt(Agent, compatRule(Situation, Type1Convert:_, Type2Convert:_),T),                                     % check related compatibility rule exists
\+ holdsAt(Agent, incompatRule(Situation, Type1Convert:_, Type2Convert:_),T),                                % check there is no applicable incompatibility rule
\+ holdsAt(Agent, compat(Situation, Type1:Value1, Type2:Value2), T),                                         % check properties are not known to be compatible
((holdsAt(Agent, propertyList(Type1Convert, List1), T), member(Value1Convert, List1)) ; number(Value1Convert)), % check property 1 category
((holdsAt(Agent, propertyList(Type2Convert, List2), T),  member(Value2Convert, List2)); number(Value2Convert)), % check property 2 category
!.                                               

% Incompatibility from incompatibility rules
holdsAt(Agent, incompat(Situation, Type1:Value1, Type2:Value2), T):- transform(Type1:Value1, Type1Convert:Value1Convert), % transform properties if applicable
transform(Type2:Value2, Type2Convert:Value2Convert), 
holdsAt(Agent, incompatRule(Situation, Type1Convert:RuleValue1, Type2Convert:RuleValue2),T),                 % applicable compatibility rule holds
(\+var(RuleValue1), (RuleValue1 = (min:Min, max:Max)) ->  
(Value1Convert >= Min, Value1Convert =< Max); extract(RuleValue1, Value1Convert)),                           % Apply numerical range or extract from list
(\+var(RuleValue2), (RuleValue2 = (min:Min, max:Max)) ->  
(Value2Convert >= Min, Value2Convert =< Max); extract(RuleValue2, Value2Convert)), !.                        % Repeat for property 2

% Compatibility from incompatibility rules

holdsAt(Agent, compat(Situation, Type1:Value1, Type2:Value2), T):- 
transform(Type1:Value1, Type1Convert:Value1Convert), transform(Type2:Value2, Type2Convert:Value2Convert),    % transform properties if applicable
holdsAt(Agent, incompatRule(Situation, Type1Convert:_, Type2Convert:_),T),                                   % check related incompatibility rule exists
\+ holdsAt(Agent, compatRule(Situation, Type1Convert:_, Type2Convert:_),T),                                  % check there is no applicable compatibility rule
\+ holdsAt(Agent, incompat(Situation, Type1:Value1, Type2:Value2), T),                                       % check properties are not known to be incompatible
((holdsAt(Agent, propertyList(Type1Convert, List1), T), member(Value1Convert, List1)) ; number(Value1Convert)), % check property 1 category
((holdsAt(Agent, propertyList(Type2Convert, List2), T),  member(Value2Convert, List2)); number(Value2Convert)), % check property 2 category
!.                               

### Example compatibilities

Let us consider some of the example compatibility rules listed above and illustrate how they enable compatibility checks.

Firstly, recall the following statement, which sets the acceptable age range for heart donations to 0 - 65 years:

`initially(monSys:compatRule(donor, organ:"Heart", age:(min:0, max:65))).`

We check whether three patients are of an acceptable age to donate a heart at time zero. The patients' dates of birth are 01/02/2005, 31/12/1972 and 13/02/1956.
The last patient is too old to donate their heart. This is confirmed when we check our **compat** fluent:

In [8]:
?- holdsAt(monSys, compat(donor, organ:"Heart", dob:(1,2,2005)),0).
?- holdsAt(monSys, compat(donor, organ:"Heart", dob:(31,12,1972)),0).
?- holdsAt(monSys, compat(donor, organ:"Heart", dob:(13,2,1956)),0).

true.
true.
false.

We can also check our **incompat** fluent which provides the opposing Boolean results:

In [9]:
?- holdsAt(monSys, incompat(donor, organ:"Heart", dob:(1,2,2005)),0).
?- holdsAt(monSys, incompat(donor, organ:"Heart", dob:(31,12,1972)),0).
?- holdsAt(monSys, incompat(donor, organ:"Heart", dob:(13,2,1956)),0).

false.
false.
true.

Recall that we declared that donors with blood type A could only donate to recipients with blood type A or AB:

In [10]:
?- holdsAt(monSys, compat(transplant, bloodType:"A", bloodType:"A"), 0).
?- holdsAt(monSys, compat(transplant, bloodType:"A", bloodType:"AB"), 0).
?- holdsAt(monSys, compat(transplant, bloodType:"A", bloodType:"B"), 0).
?- holdsAt(monSys, compat(transplant, bloodType:"A", bloodType:"O"), 0).

true.
true.
false.
false.

Recall that we set livers to be compatible with all of our recognised medical conditions, except heat disease, using the following line of code:

`initially(monSys:incompatRule(donor, organ:"Liver", condition:"Heart Disease")).`

And that the following conditions are recognised by **monSys**: HIV, Hep-B, Hep-C, Heart Disease, Severe Asthma, and Chronic Pulmonary Disease.

As "Severe Asthma" is on our recognised list of conditions, we find that it is compatible with liver donations, and heart disease is not:

In [11]:
?- holdsAt(monSys, compat(donor, organ:"Liver", condition:"Severe Asthma"), 0).
?- holdsAt(monSys, compat(donor, organ:"Liver", condition:"Heart Disease"), 0).
?- holdsAt(monSys, incompat(donor, organ:"Liver", condition:"Severe Asthma"), 0).
?- holdsAt(monSys, incompat(donor, organ:"Liver", condition:"Heart Disease"), 0).

true.
false.
false.
true.

If we test a condition that we have not previously declared to be recognised by **monSys**, both compatibility and incompatibility checks return false, as its compatibility status with liver donation is unknown. Note that if a compatibility check returning false, does not necessarily mean incompatibility!

In [12]:
?- holdsAt(monSys, compat(donor, organ:"Liver", condition:"made up disease"), 0).
?- holdsAt(monSys, incompat(donor, organ:"Liver", condition:"made up disease"), 0).

false.
false.

While the current predicates only allow us to check the compatibility of two properties at a time, this could be expanded to include far more complex rules.

### Example Narrative

Having exhibited the compatibility rules working for simple compatibility checks, we now wish to show that they function in our wider organ donation scenario.

At time period zero, Otago hospital sends a waiting list add request to **monSys** for patient 104. This patient is in need of a heart, has asthma, blood type B and was born on the 1st of April 2015.

At time period three, Wellington hospital sends a donation offer to **monSys** for patient 201. They have five organs suitable for donation, blood type A, have hepatitis C and were born on the 10th of November 1978.

At time period four, **monSys** matches the waiting list record for patient 104 from Otago to the donor offer from patient 201 from Wellington. Note that the donor and the recipient have incompatible blood types for a donation.

In [13]:
% File: narrative.pl
happensAtNarrative("Otago":send(monSys, waitAddReq(104, ["heart"], [bloodType:"B"])),0).
% dob:(01,04,2015), condition:"Asthma"
% dob:(10,11,1950), condition:"HepC"
happensAtNarrative("Wellington": send(monSys, donorOffer(201, [heart, kidney, liver, lungs, pancreas], [bloodType:"A"])),3).
happensAtNarrative(monSys:match(701, "Otago", 104, "Wellington", 201, "Auckland", ["heart"], [donorCondition:"HepC"]),5).

We add an expectation rule stating that when a match is made, **monSys** expects that there will not be any properties of the donor and the recipient which are incompatible.

In [14]:
% File: compatibilityExpectations.pl
initially(monSys:exp_rule(and([happ(match(_, RecipientHospital, RecipientID, DonorHospital, DonorID, Location, Organs, Notes)), 
waiting(RecipientID, RecipientHospital, RecipientOrgans, RecipientDetails, _), donorOffered(Hospital, DonorID, DonorOrgans, DonorDetails)]), 
not((and([incompat(transplant, DonorProperty, RecipientProperty), condition(memberchk(DonorProperty, DonorDetails)), 
condition(memberchk(RecipientProperty, RecipientDetails))]))), dependent, "Matched donor and recipient are not incompatible")).


## Discussion of incomplete functionality:

At time period 5, the match occurs, but we find that currently, our expectation rule leads to a fulfilment occurring, even though the donor and the recipient's blood types are not compatible.

In [15]:
?- run(10).
?- happensAt(monSys, Event, 5).

true.
Event = fulf(and([Functor(14512269,1,match(701, b'Otago', 104, b'Wellington', 201, b'Auckland', [b'heart'], [Functor(188685,2,donorCondition,b'HepC')])), Functor(14594701,5,104,b'Otago',[b'heart'],[Functor(188685,2,bloodType,b'B')],1), Functor(14631437,4,b'Wellington',201,[Atom('722053'), Atom('722181'), Atom('722309'), Atom('722437'), Atom('722565')],[Functor(188685,2,bloodType,b'A')])]), not(and([Functor(15270285,3,transplant,:(bloodType, b'A'),:(bloodType, b'B')), Functor(14487693,1,memberchk(:(bloodType, b'A'), [Functor(188685,2,bloodType,b'A')])), Functor(14487693,1,memberchk(:(bloodType, b'B'), [Functor(188685,2,bloodType,b'B')]))])), 5, not(and([Functor(15270285,3,transplant,:(bloodType, b'A'),:(bloodType, b'B')), Functor(14487693,1,memberchk(:(bloodType, b'A'), [Functor(188685,2,bloodType,b'A')])), Functor(14487693,1,memberchk(:(bloodType, b'B'), [Functor(188685,2,bloodType,b'B')]))])), dependent, b'Matched donor and recipient are not incompatible') ;
Event = match(701, b

This occurs because general compatibility or incompatibility queries, such as the one shown below, are not currently supported to list all compatibilities or incompatibilities. Note that even though we can directly check if blood types A and B are incompatible, if we ask for all combinations of known incompatible transplant properties, we do not get any results. This means that our expectation rule is unable to reliably check if incompatibility and compatibility fluents evaluate to true at a given time period.

In [24]:
?- holdsAt(monSys, incompat(transplant, Property1, Property2), 5).
?- holdsAt(monSys, incompat(transplant, bloodType:"A", bloodType:"B"), 5).

false.
true.

When we evaluate the `incompat` fluent directly, our expression correctly evaluates to false. However, when we use a variable argument, our expression evaluation is incorrect. 

In [31]:
?- eval(monSys, not(incompat(transplant, bloodType:"A", bloodType:"B")), 5, Boolean).
?- eval(monSys, not(and([member(Property, [bloodType:"B"]), incompat(transplant, bloodType:"A", Property)])), 5, Boolean).

Boolean = false .
Property = _1898, Boolean = true .

Once this issue is addressed, we should be able to successfully incorporate compatibility and incompatibility fluents into expectations about donors, recipients and matches. As well as **monSys** expecting that the donor and the recipient are compatible, **monSys** or potentially individual hospitals could expect that either the donor or the recipient which are externally sourced are suitable to be involved in a transplant.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32f94018-a4da-40ef-8c9f-8983d73811c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>